In [1]:
import time
import os
import numpy as np
import pandas as pd
import matrix_code

from scipy import sparse, linalg
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import sample
from itertools import combinations

In [2]:
dir_path = "/mnt/lab_data/kundaje/zijzhao/"
cell_annotate = pd.read_csv(os.path.join(dir_path, "cell_annotate.csv"))

In [3]:
gene_annotate = pd.read_csv(os.path.join(dir_path, "gene_annotate.csv"))

In [4]:
# we only choose 'Neural tube and notochord trajectory' 632188 cells
NTN_traj_annotate = cell_annotate.loc[cell_annotate['Main_trajectory'] == 
                                      'Neural tube and notochord trajectory'].reset_index(drop=True)

In [5]:
# preprocess the feature matrix
count = sparse.load_npz(os.path.join(dir_path, "organogenesis_mouse.npz"))

In [6]:
ntn_ind = cell_annotate.index[cell_annotate['Main_trajectory'] == 
                                      'Neural tube and notochord trajectory'].to_numpy()
X = count[ntn_ind,:]

In [7]:
# construct X
## Preoprocess I: only keep protein coding genes
procoding_ind = gene_annotate.index[gene_annotate['gene_type']=='protein_coding'].to_numpy()
gene_annotate = gene_annotate.loc[gene_annotate['gene_type']=='protein_coding'].reset_index(drop=True)
X = X.transpose()[procoding_ind, :].transpose()

In [8]:
## Preprocess II: filter out no expression genes
zeroread_ind = np.where(X.sum(axis=0)!=0)[1]
gene_annotate = gene_annotate.iloc[zeroread_ind].reset_index(drop=True)
X = X.transpose()[zeroread_ind, :].transpose() 

In [9]:
## Preprocess III: filter out mito genes (genes starting with 'mt-')
gene_name = np.array([1 if 'mt-' in i else 0 for i in np.array(gene_annotate['gene_short_name'])])
nonmt_ind = np.where(gene_name == 0)[0]
gene_annotate = gene_annotate.iloc[nonmt_ind].reset_index(drop=True)
X = X.transpose()[nonmt_ind, :].transpose() 

In [10]:
# helper function
def perReadScore(X, X_norm, axis=0):
    """
    Helper function for preprocess.
    sums of squares of normalized column entries / num of reads
    """

    num_reads = X.sum(axis=0)
    return X_norm.power(2).sum(axis=axis)/num_reads 

In [11]:
## Preprocess IV: Normalization & filter out low expression variance genes
## only keep genes with variance above 10% quantile
X_norm = matrix_code.deviance_residuals(X)
score = perReadScore(X, X_norm)
quantile = np.quantile(score, 0.1)
abovequant_ind = np.where(score >= quantile)[1]
gene_annotate = gene_annotate.iloc[abovequant_ind].reset_index(drop=True)
X_norm = X_norm.transpose()[abovequant_ind, :].transpose()

In [29]:
X_norm

<632188x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 183942311 stored elements in Compressed Sparse Row format>

In [13]:
import scanpy as sc, anndata

In [ ]:
itime = time.time()
nbrs = sc.pp.neighbors(anndata.AnnData(X_norm), use_rep='X')
print(time.time() - itime)

In [38]:
test_nbrs.uns['neighbors']['connectivities']

AttributeError: 'NoneType' object has no attribute 'uns'